In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 20 13:38:16 2023

@author: Lenovo
"""

import time
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch

print("是否使用GPU训练：{}".format(torch.cuda.is_available()))    #打印是否采用gpu训练
if torch.cuda.is_available:
    print("GPU名称为：{}".format(torch.cuda.get_device_name()))  #打印相应的gpu信息
normalize=transforms.Normalize(mean=[5,5,5], std=[5,5,5]) #规范化
transform=transforms.Compose([transforms.Resize((64, 64)),transforms.ToTensor(),normalize]) #数据处理
dataset_train=ImageFolder('add_label/label_picture/train',transform=transform)     #训练数据集
# print(dataset_tran[0])
dataset_test=ImageFolder('add_label/label_picture/test',transform=transform)     #验证或测试数据集
dataset_valid=ImageFolder('add_label/label_picture/valid',transform=transform)

# print(dataset_train.classer)#返回类别
print(dataset_train.class_to_idx)                               #返回类别及其索引
# print(dataset_train.imgs)#返回图片路径
print(dataset_test.class_to_idx)
print(dataset_valid.class_to_idx)
train_data_size=len(dataset_train)                              #放回数据集长度
test_data_size=len(dataset_test)
valid_data_size=len(dataset_valid)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))
print("验证数据集的长度为：{}".format(test_data_size))
#torch自带的标准数据集加载函数
dataloader_train=DataLoader(dataset_train,batch_size=4,shuffle=True,num_workers=0,drop_last=True)
dataloader_test=DataLoader(dataset_test,batch_size=4,shuffle=True,num_workers=0,drop_last=True)
dataloader_valid=DataLoader(dataset_valid,batch_size=4,shuffle=True,num_workers=0,drop_last=True)
#模型加载
model_ft=models.resnet50(pretrained=True)#使用迁移学习，加载预训练权重
# print(model_ft)

in_features=model_ft.fc.in_features
model_ft.fc=nn.Sequential(nn.Linear(in_features,36),nn.Linear(36,4))#将最后的全连接改为（36，6），使输出为六个小数，对应四个树种的置信度


model_ft=model_ft.cuda()#将模型迁移到gpu

#pytorch2.0编译模型部分
# model_ft=torch.compile(model_ft)

#优化器
loss_fn=nn.CrossEntropyLoss()

loss_fn=loss_fn.cuda()  #将loss迁移到gpu
learn_rate=0.0001       #设置学习率
optimizer=torch.optim.SGD(model_ft.parameters(),lr=learn_rate,momentum=0.001)#可调超参数


total_train_step=0
total_test_step=0
total_valid_step=0

epoch=50              #迭代次数
writer=SummaryWriter("logs")
best_acc=-1
ss_time=time.time()

for i in range(epoch):
    start_time = time.time()
    print("--------第{}轮训练开始---------".format(i+1))
    model_ft.train()
    for data in dataloader_train:
        imgs,targets=data
        imgs=imgs.cuda()
        targets=targets.cuda()
        outputs=model_ft(imgs)
        loss=loss_fn(outputs,targets)

        optimizer.zero_grad()   #梯度归零
        loss.backward()         #反向传播计算梯度
        optimizer.step()        #梯度优化

        total_train_step=total_train_step+1
        if total_train_step%100==0:#一轮时间过长可以考虑加一个
            end_time=time.time()
            print("使用GPU训练100次的时间为：{}".format(end_time-start_time))
            print("训练次数：{},loss:{}".format(total_train_step,loss.item()))

    model_ft.eval()
    
    total_test_loss=0
    total_test_accuracy=0
    
    total_train_loss=0
    total_train_accuracy=0

    total_valid_loss = 0
    total_valid_accuracy = 0
    with torch.no_grad():       #验证数据集时禁止反向传播优化权重
      #记录训练集精度和损失值
        for data in dataloader_train:
            imgs,targets=data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs=model_ft(imgs)
            train_loss=loss_fn(outputs,targets)
            total_train_loss=total_train_loss+train_loss.item()
            train_accuracy=(outputs.argmax(1)==targets).sum()
            total_train_accuracy=total_train_accuracy+train_accuracy
        print("整体训练集上的loss：{}(越小越好,与上面的loss无关此为测试集的总loss)".format(total_train_loss))
        print("整体训练集上的正确率：{}(越大越好)".format(total_train_accuracy / len(dataset_train)))
       
        for data in dataloader_test:
            imgs,targets=data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs=model_ft(imgs)
            test_loss=loss_fn(outputs,targets)
            total_test_loss=total_test_loss+test_loss.item()
            test_accuracy=(outputs.argmax(1)==targets).sum()
            total_test_accuracy=total_test_accuracy+test_accuracy
        print("整体测试集上的loss：{}(越小越好,与上面的loss无关此为测试集的总loss)".format(total_test_loss))
        print("整体测试集上的正确率：{}(越大越好)".format(total_test_accuracy / len(dataset_test)))

        for data in dataloader_valid:
            imgs, targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs = model_ft(imgs)
            valid_loss = loss_fn(outputs, targets)
            total_valid_loss = total_valid_loss + valid_loss.item()
            valid_accuracy = (outputs.argmax(1) == targets).sum()
            total_valid_accuracy = total_valid_accuracy + valid_accuracy
        print("整体验证集上的loss：{}(越小越好,与上面的loss无关此为测试集的总loss)".format(total_valid_loss))
        print("整体验证集上的正确率：{}(越大越好)".format(total_valid_accuracy / len(dataset_valid)))

        # 记录到 TensorBoard
        writer.add_scalar("Train Accuracy", total_train_accuracy / len(dataset_train), i)
        writer.add_scalar("Train Loss", total_train_loss, i)
        writer.add_scalar("Test Accuracy", total_test_accuracy/len(dataset_test), i)
        writer.add_scalar("Test Loss", total_test_loss, i)
        writer.add_scalar("Valid Accuracy", total_valid_accuracy / len(dataset_valid), i)
        writer.add_scalar("Valid Loss", total_valid_loss, i)

        total_test_step = total_test_step + 1
        if total_test_accuracy > best_acc:   #保存迭代次数中最好的模型
            print("已修改模型")
            best_acc = total_test_accuracy
            torch.save(model_ft.state_dict(), "best_model.pth")      #只保留权重的参数即可 
# # 绘制训练和测试精度和损失曲线
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(total_train_loss, label='Train Loss')
# plt.plot(total_test_loss, label='Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()


# plt.subplot(1, 2, 2)
# plt.plot(total_train_accuracy / len(dataset_train), label='Train Accuracy')
# plt.plot(total_test_accuracy/len(dataset_test), label='Test Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()
# plt.show()
ee_time=time.time()
zong_time=ee_time-ss_time
print("训练总共用时:{}h:{}m:{}s".format(int(zong_time//3600),int((zong_time%3600)//60),int(zong_time%60))) #打印训练总耗时
writer.close()

是否使用GPU训练：True
GPU名称为：NVIDIA GeForce RTX 4060 Ti
{'fir': 0, 'pine': 1, 'spruce': 2, 'trembling aspen': 3}
{'fir': 0, 'pine': 1, 'spruce': 2, 'trembling aspen': 3}
{'fir': 0, 'pine': 1, 'spruce': 2, 'trembling aspen': 3}
训练数据集的长度为：597
测试数据集的长度为：85
验证数据集的长度为：85


/venv/main/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/venv/main/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


--------第1轮训练开始---------
使用GPU训练100次的时间为：3.6568214893341064
训练次数：100,loss:1.3547441959381104
整体训练集上的loss：202.92169106006622(越小越好,与上面的loss无关此为测试集的总loss)
整体训练集上的正确率：0.30485761165618896(越大越好)
整体测试集上的loss：28.36172652244568(越小越好,与上面的loss无关此为测试集的总loss)
整体测试集上的正确率：0.29411765933036804(越大越好)
整体验证集上的loss：56.679781794548035(越小越好,与上面的loss无关此为测试集的总loss)
整体验证集上的正确率：0.3294117748737335(越大越好)
已修改模型
--------第2轮训练开始---------
使用GPU训练100次的时间为：1.694918155670166
训练次数：200,loss:1.5125455856323242
整体训练集上的loss：201.03995007276535(越小越好,与上面的loss无关此为测试集的总loss)
整体训练集上的正确率：0.3366834223270416(越大越好)
整体测试集上的loss：28.797962188720703(越小越好,与上面的loss无关此为测试集的总loss)
整体测试集上的正确率：0.3294117748737335(越大越好)
整体验证集上的loss：56.9304895401001(越小越好,与上面的loss无关此为测试集的总loss)
整体验证集上的正确率：0.30588236451148987(越大越好)
已修改模型
--------第3轮训练开始---------
使用GPU训练100次的时间为：0.05579423904418945
训练次数：300,loss:1.4746609926223755
使用GPU训练100次的时间为：2.9517948627471924
训练次数：400,loss:1.2638483047485352
整体训练集上的loss：198.63195610046387(越小越好,与上面的loss无关此为测试集的总loss)
整体训练集上的正确率：0